# OOP (2024 Fall) 실습02: 캡슐화 원리가 적용된 방향그래프
- 이름: 전세영
- 학번: 20210513

여기 홍길동, 99999999 대신 본인의 이름, 학번 작성

import javax.imageio.*;

var mmgraph = """
graph LR;
    서울 --> 원주 --> 평창 --> 강릉
    원주 --> 안동
    서울 --> 천안 --> 대전 & 공주
    대전 --> 대구 --> 부산
    공주 --> 익산 --> 광주 --> 목포
    대전 --> 익산 --> 전주 --> 여수
""";

display(ImageIO.read(new URL("https://mermaid.ink/img/"+Base64.getEncoder().encodeToString(mmgraph.getBytes()))));

## 방향그래프 레코드
노드가 String인 인접리스트 표현을 구성요소로 포함하는 레코드의 정의를 캡슐화 원리가 잘 지켜지도록 수정/완성해 보라.

adjList는 실습01에서 작성했던 routeMap으로 초기화한다고 보면 된다.

In [23]:
import java.util.*;

// Graph 레코드 정의
public record Graph(Map<String, List<String>> adjList) {
    
    // 기본 생성자: 불변 Map과 List로 변환하여 캡슐화 보장
    public Graph(Map<String, List<String>> adjList) {
        // adjList가 null이 아닌지 확인하고, 내부적으로 불변성을 가진 Map으로 변환
        this.adjList = Collections.unmodifiableMap(
            adjList.entrySet().stream()
            .collect(Collectors.toMap(
                Map.Entry::getKey, 
                e -> Collections.unmodifiableList(new ArrayList<>(e.getValue()))
            ))
        );
    }

    // orig 노드로부터 직접 연결된 노드 리스트를 반환 (orig가 존재하지 않으면 null 반환)
    public List<String> adjFrom(String orig) {
        // 유효한 orig인지 확인하고, 있으면 해당 리스트의 복제본을 반환하여 외부 변경 방지
        List<String> adjacentNodes = adjList.get(orig);
        return adjacentNodes == null ? null : List.copyOf(adjacentNodes);
    }

    // 선택사항: orig에서 dest까지 경로가 존재하는지 확인하는 메서드 (DFS 또는 BFS 사용 가능)
    public boolean hasPath(String orig, String dest) {
        if (!adjList.containsKey(orig) || !adjList.containsKey(dest)) {
            return false; // orig 또는 dest가 그래프에 없는 경우
        }
        // DFS로 경로 탐색
        Set<String> visited = new HashSet<>();
        return dfs(orig, dest, visited);
    }

    // 깊이 우선 탐색(DFS)을 사용하여 경로를 찾는 메서드
    private boolean dfs(String current, String dest, Set<String> visited) {
        if (current.equals(dest)) {
            return true;
        }
        visited.add(current);
        List<String> neighbors = adjList.get(current);
        if (neighbors != null) {
            for (String neighbor : neighbors) {
                if (!visited.contains(neighbor) && dfs(neighbor, dest, visited)) {
                    return true;
                }
            }
        }
        return false;
    }
}


#### 코드에서 어떤 부분을 신경써서 캡슐화 원리가 잘 지켜지도록 작성했는지 되도록 짧게 설명하라. (200자 넘지 않게)

노드의 인접 리스트를 반환할 때 복사본을 제공하여 원본 데이터에 대한 직접적인 접근을 방지함으로써, 캡슐화를 강화했습니다.

#### 그래프 객체 생성 및 활용
실습01에서처럼 위의 하행선 철도 노선도를 자바의 Map과 List를 활용한 인접리스트(adjacencty list) 데이터 구조로 옮겨 `Graph` 레코드의 인스턴스를 생성해 `routeGraph` 변수에 저장해 보라.

In [24]:
// routeMap은 지난 lab01 내용
var routeMap = new HashMap< String, List<String> >() {{ // modifiable map with modifiable values 
    put("서울", new ArrayList<>(List.of("원주","천안")));
    put("원주", new ArrayList<>(List.of("평창","안동")));  put("안동", new ArrayList<>());
    put("평창", new ArrayList<>(List.of("강릉")));        put("강릉", new ArrayList<>());
    put("천안", new ArrayList<>(List.of("대전","공주")));
    put("대전", new ArrayList<>(List.of("대구","익산")));
    put("대구", new ArrayList<>(List.of("부산")));        put("부산", new ArrayList<>());
    put("공주", new ArrayList<>(List.of("익산")));
    put("익산", new ArrayList<>(List.of("광주","전주")));
    put("광주", new ArrayList<>(List.of("목포")));        put("목포", new ArrayList<>());
    put("전주", new ArrayList<>(List.of("여수")));        put("여수", new ArrayList<>());
}};

System.out.println(routeMap);

{강릉=[], 전주=[여수], 익산=[광주, 전주], 서울=[원주, 천안], 부산=[], 안동=[], 평창=[강릉], 대전=[대구, 익산], 공주=[익산], 원주=[평창, 안동], 목포=[], 여수=[], 천안=[대전, 공주], 대구=[부산], 광주=[목포]}


In [25]:
import java.util.*;
import java.util.stream.Collectors;

// Graph 레코드 정의
public record Graph(Map<String, List<String>> adjList) {

    // 기본 생성자: 불변 Map과 List로 변환하여 캡슐화 보장
    public Graph(Map<String, List<String>> adjList) {
        // adjList가 null이 아닌지 확인하고, 내부적으로 불변성을 가진 Map으로 변환
        if (adjList == null) {
            throw new IllegalArgumentException("인접 리스트는 null일 수 없습니다.");
        }
        
        this.adjList = Collections.unmodifiableMap(
            adjList.entrySet().stream()
            .collect(Collectors.toMap(
                Map.Entry::getKey, 
                e -> Collections.unmodifiableList(new ArrayList<>(e.getValue()))
            ))
        );
    }

    // orig 노드로부터 직접 연결된 노드 리스트를 반환 (orig가 존재하지 않으면 빈 리스트 반환)
    public List<String> adjFrom(String orig) {
        // orig 노드가 존재하지 않을 경우 빈 리스트를 반환
        return adjList.getOrDefault(orig, Collections.emptyList());
    }
}


그리고 adjFrom 메소드를 호출했을 때 크기가 0, 1, 2인 리스트를 리턴하는 경우 및 null을 리턴하는 경우를 작성해 보라.

In [26]:
System.out.println(new Graph(Map.of("서울", List.of("원주", "천안"))).adjFrom("서울")); // [원주, 천안]


[원주, 천안]


In [27]:
System.out.println(new Graph(Map.of("원주", List.of("평창"))).adjFrom("원주")); // [평창]


[평창]


In [28]:
System.out.println(new Graph(Map.of("평창", List.of())).adjFrom("평창")); // []


[]


In [29]:
System.out.println(new Graph(Map.of("안동", List.of())).adjFrom("안동")); // []


[]


In [30]:
System.out.println(new Graph(Map.of()).adjFrom("존재하지 않는 노드")); // []


[]


In [31]:
// System.out.println( routeGraph.adjFrom("???") );